<a href="https://colab.research.google.com/github/icapucap/SE_project/blob/main/vgg_ninv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD,Adam
from keras.layers import Activation, AveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.initializers import glorot_normal
from keras.utils import np_utils
from keras import backend as K
from keras.datasets import cifar10
from keras.callbacks import  EarlyStopping, LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.optimizers.schedules import ExponentialDecay
from keras.metrics import AUC
from keras.utils.vis_utils import plot_model

In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.init(project="vgg-cifar10_ninv2")

In [ ]:
# data loading and preprocessing

batch_size = 128
epochs = 50
learning_rate = 0.05
lr_decay = 1e-6
lr_drop = 20
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train  /= 255
x_test /= 255

class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
               
datagen = ImageDataGenerator(
            featurewise_center=False, 
            samplewise_center=False,  
            featurewise_std_normalization=False,  
            samplewise_std_normalization=False,  
            zca_whitening=False,  
            rotation_range=15,  
            width_shift_range=0.1,  
            height_shift_range=0.1,  
            horizontal_flip=True, 
            vertical_flip=False)  
      
datagen.fit(x_train)


In [ ]:
BATCH_NORM = True
num_classes=10


In [ ]:
def base_model_with_ninv2():

    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same', input_shape=x_train.shape[1:], name='block1_conv1'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(64, (1, 1), padding='same', name='block1_conv2nin1'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(64, (1, 1), padding='same', name='block1_conv2nin2'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(64, (1, 1), padding='same', name='block1_conv2nin3'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(AveragePooling2D((2, 2), strides=(2, 2), name='block1_pool'))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same', name='block2_conv1' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(128, (1, 1), padding='same', name='block2_conv2nin1' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(128, (1, 1), padding='same', name='block2_conv2nin2' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(64, (1, 1), padding='same', name='block2_conv2nin3'))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(AveragePooling2D((2, 2), strides=(2, 2), name='block2_pool'))
    model.add(Dropout(0.2))

    model.add(Conv2D(256, (3, 3), padding='same', name='block3_conv1' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(256, (1, 1), padding='same', name='block3_conv2nin1' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(256, (1, 1), padding='same', name='block3_conv3nin2' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(256, (1, 1), padding='same', name='block3_conv4nin3' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(AveragePooling2D((2, 2), strides=(2, 2), name='block3_pool'))
    model.add(Dropout(0.2))

    model.add(Conv2D(512, (3, 3), padding='same', name='block4_conv1' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, (1, 1), padding='same', name='block4_conv2nin1' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, (1, 1), padding='same', name='block4_conv3nin2' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, (1, 1), padding='same', name='block4_conv4nin3' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(AveragePooling2D((2, 2), strides=(2, 2), name='block4_pool'))
    model.add(Dropout(0.2))

    model.add(Conv2D(512, (3, 3), padding='same', name='block5_conv1' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, (1, 1), padding='same', name='block5_conv2nin1' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, (1, 1), padding='same', name='block5_conv3nin2' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(512, (1, 1), padding='same', name='block5_conv4nin3' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))

    model.add(Conv2D(10, (1, 1), padding='same', name='block5_convlast' ))
    model.add(BatchNormalization()) if BATCH_NORM else None
    model.add(Activation('relu'))


    model.add(GlobalAveragePooling2D())

    model.add(Activation('softmax'))

    # sgd = SGD(lr=learning_rate, decay=lr_decay, momentum=0.75, nesterov=True)
    lr_schedule = ExponentialDecay(
    initial_learning_rate=5e-4,
    decay_steps=10000,
    decay_rate=0.85)
    optimizer = Adam(learning_rate=lr_schedule)
    
    model.compile(loss='categorical_crossentropy', optimizer=optimizer,metrics=['accuracy', AUC()])
    return model


In [ ]:
# prints model summary and also stores the same as an image in png format

cnn_n = base_model_with_ninv2()
cnn_n.summary()
plot_model(cnn_n, to_file='ninv2.png', show_shapes=True, show_layer_names=True)

In [ ]:
cnn_n.fit_generator(datagen.flow(x_train, y_train,
                                         batch_size=batch_size),
                            steps_per_epoch=x_train.shape[0] // batch_size,
                            epochs=epochs,
                            validation_data=(x_test, y_test),callbacks=[WandbCallback(data_type="image", validation_data=(x_test, y_test), labels=class_names),
                               EarlyStopping(monitor='val_accuracy',patience=5, restore_best_weights=True)],verbose=2)
#cnn_n.save_weights('cifar10vgg.h5')

In [ ]:


from sklearn.metrics import classification_report, confusion_matrix
Y_pred = cnn_n.predict(x_test, verbose=2)
y_pred = np.argmax(Y_pred, axis=1)

for ix in range(10):
    print(ix, confusion_matrix(np.argmax(y_test,axis=1),y_pred)[ix].sum())
cm = confusion_matrix(np.argmax(y_test,axis=1),y_pred)
print(cm)

# Visualizing of confusion matrix
import seaborn as sn
import pandas  as pd


df_cm = pd.DataFrame(cm, range(10),
                  range(10))
plt.figure(figsize = (10,7))
sn.set(font_scale=1.4)#for label size
sn.heatmap(df_cm, annot=True,annot_kws={"size": 12})# font size
plt.show()